ファイル一覧を確認する

In [30]:
ls

FBA-Yeast.ipynb                 iMM904Test.txt
iMM904_181029aminoacid_def.txt  pyfba181121forSuntory.py
iMM904_181029aminoacid.txt      simulation_suntory181028.py
iMM904_181029_def.txt           simulation_suntory181029aminoacid.py
iMM904_181029.txt               simulation_suntory181029.py
iMM904_181030aminoacid_def.txt  Untitled.ipynb
iMM904_181030aminoacid.txt


モデル定義の読み込み

In [38]:
import re, csv
import pulp

class MetabolicModel:
    def __init__(self, deffilename, modelfilename, mode = "normal"):
        format = "text"
        output = "normal"

        self.separator = {}
        self.constraint = {}
        self.optstep = {}
        self.inconstant = {}
        self.inconstant_array = []
        self.inconstant_onlyincrease = {}
        self.metabolites = {}
        self.reactions = {}
        self.reactionid_array = []
        self.maboliteid_array = []
        with open(deffilename, 'r') as f:
            if format == "text":
                reader = csv.reader(f, delimiter='\t')
            elif format == "csv":
                reader = csv.reader(f, dialect='excel')
            else:
                print("Unknown format!")
                f.close()
                return False

            for i, row in enumerate(reader):

                if len(row) == 0:
                    continue
                if "#" == row[0][0]:
                    continue
                row = [item for item in row if "#" not in item]
                # Remove '' from row
                row = list(filter(lambda s:s != '', row))

                if len(row) < 1:
                    continue
                row0 = row[0].replace(" ", "")
                if row0 == "separator_reversible":
                    separator_reversible = row[1]
                    separator_reversible = re.sub(r'^\s+',"",separator_reversible)
                    separator_reversible = re.sub(r'\s+$',"",separator_reversible)
                    self.separator["reversible"] = separator_reversible
                    continue
                if row0 == "separator_onedirection":
                    separator_onedirection = row[1]
                    separator_onedirection = re.sub(r'^\s+',"",separator_onedirection)
                    separator_onedirection = re.sub(r'\s+$',"",separator_onedirection)
                    self.separator["onedirection"] = separator_onedirection
                    continue
                if row0 == "separator_plus":
                    separator_plus = row[1]
                    separator_plus = re.sub(r'^\s+',"",separator_plus)
                    separator_plus = re.sub(r'\s+$',"",separator_plus)
                    self.separator["plus"] = separator_plus
                    continue
                if row0 == "optstep":
                    optstep = row[1]
                    optstep = re.sub(r'^\s+',"",optstep)
                    optstep = re.sub(r'\s+$',"",optstep)
                    optstep = self.check_reactionname(optstep)
                    if len(row) > 3:
                        self.optstep[optstep] = float(row[2])
                    else:
                        self.optstep[optstep] = 1.0


                    continue
                if row0 == "constraint":
                    temp = re.split(r'\s+\=\s+',row[1])
                    reaction = temp[0]
                    value = temp[1]
                    reaction = re.sub(r'^\s+',"",reaction)
                    reaction = re.sub(r'\s+$',"",reaction)
                    reaction = self.check_reactionname(reaction)
                    self.constraint[reaction] = value
                    continue

                if row0 == "inconstant_start":
                    mode = "inconstant"
                    continue
                if row0 == "inconstant_end":
                    mode = "normal"
                    continue
                if row0 == "inconstant_onlyincrease_start":
                    mode = "inconstant_onlyincrease"
                    continue
                if row0 == "inconstant_onlyincrease_end":
                    mode = "normal"
                    continue

                if mode == "inconstant":
                    if output == "debug":
                        print(row)
                    if len(row) < 1:
                        continue
                    rid = row[0].replace(" ", "")
                    rid = self.check_metabolitename(rid)
                    if rid == "":
                        continue

                    self.inconstant[rid] = 1
                    self.inconstant_array.append(rid)
                if mode == "inconstant_onlyincrease":
                    if output == "debug":
                        print(row)
                    if len(row) < 1:
                        continue
                    rid = row[0].replace(" ", "")
                    if rid == "":
                        continue
                    rid = re.sub(r'\_',"",rid)
                    rid = self.check_metabolitename(rid)
                    self.inconstant_onlyincrease[rid] = 1

        #
        #化合物名を収集する。
        #
        reactionid = 0

        with open(modelfilename, 'r') as f:
            if format == "text":
                reader = csv.reader(f, delimiter='\t')
            elif format == "csv":
                reader = csv.reader(f, dialect='excel')
            else:
                print("Unknown format!")
                f.close()
                return False

            for i, row in enumerate(reader):
                if len(row) < 6:
                    continue
                rid = row[0].replace('"', "")		#ID
                rid = self.check_reactionname(rid)


                reaction = row[5].replace('"', "") #	反応式
                flag = row[1].replace('"', "")
                group = row[2].replace('"', "")	#反応の説明
                lb = row[3].replace('"', "") #下限
                ub = row[4].replace('"', "") #上限
                reaction = re.sub(r'^\s+',"",reaction)
                reaction = re.sub(r'\s+$',"",reaction)
                if flag == "0":
                    continue
                pat = re.compile(r'\s+{0}\s+|\s+{1}\s+'.format(separator_onedirection, separator_reversible))
                reactions = re.split(pat, reaction)
                if output == "debug":
                    print(rid, reaction, group, separator_onedirection, separator_reversible)
                if not len(reactions) == 2:
                    continue
                substrate = reactions[0]
                product = reactions[1]
                substrates = re.split(r'\s+\+\s+', substrate)
                products = re.split(r'\s+\+\s+', product)
                self.reactions[rid] = {}
                self.reactions[rid]["reaction"] = reaction
                self.reactions[rid]["group"] = group
                self.reactions[rid]["lb"] = lb
                self.reactions[rid]["ub"] = ub
                self.reactions[rid]["metabolites"] = {}
                self.reactions[rid]["reactionid"] = reactionid

                reactionid = reactionid + 1
                self.reactionid_array.append(rid)

                for compound in substrates:
                    cps =  re.split(r' +', compound)
                    #
                    # 1.1348 13BDglcn[c]
                    #
                    if re.match(r'^[0-9]+\.*[0-9]*$', cps[0]):
                        compound = cps[1]
                        compound = self.check_metabolitename(compound)
                        self.reactions[rid]["metabolites"][compound] = float(cps[0]) * -1.0
                    #
                    # (1.1348) 13BDglcn[c]
                    #
                    elif re.match(r'^\([0-9]+\.*[0-9]*\)$', cps[0]):
                        compound = cps[1]
                        compound = self.check_metabolitename(compound)
                        temp = str(cps[0])
                        temp = temp.replace('(','')
                        temp = temp.replace(')','')
                        self.reactions[rid]["metabolites"][compound] = float(temp) * -1.0
                    else:
                        compound = self.check_metabolitename(compound)
                        self.reactions[rid]["metabolites"][compound] = -1.0
                    self.metabolites[compound] = {"metaboliteid": 1, "reactions":{}}

                for compound in products:
                    cps =  re.split(r' +', compound)
                    if re.match(r'^[0-9]+\.*[0-9]*$', cps[0]):
                        compound = cps[1]
                        compound = self.check_metabolitename(compound)
                        self.reactions[rid]["metabolites"][compound] = float(cps[0]) * 1.0
                    #
                    # (1.1348) 13BDglcn[c]
                    #
                    elif re.match(r'^\([0-9]+\.*[0-9]*\)$', cps[0]):
                        compound = cps[1]
                        compound = self.check_metabolitename(compound)
                        temp = str(cps[0])
                        temp = temp.replace('(','')
                        temp = temp.replace(')','')
                        self.reactions[rid]["metabolites"][compound] = float(temp) * 1.0
                    else:
                        compound = self.check_metabolitename(compound)
                        self.reactions[rid]["metabolites"][compound] = 1.0
                    self.metabolites[compound] = {"metaboliteid": 1, "reactions":{}}
    def check_metabolitename(self, metid):
        if re.match(r'^[0-9]+', metid):
            metid = ""+metid
        return metid
    def check_reactionname(self, rid):
        if re.match(r'^[0-9]+', rid):
            rid = "R"+rid
        rid = rid.replace('-', "_")		#ID
        return rid
    def solve(self, output = "normal"):
        #化合物名の整理
        self.maboliteid_array = list(sorted(self.metabolites.keys()))
        for i, met in enumerate(self.maboliteid_array):
            self.metabolites[met]["metaboliteid"] = i
        #化合物名の整理
        for i, reaction in enumerate(self.reactionid_array):
            for metabolite in self.reactions[reaction]["metabolites"]:
                self.metabolites[metabolite]["reactions"][reaction] =  self.reactions[reaction]["metabolites"][metabolite]
        #
        # PuLPのモデル構築
        #
        self.problem = pulp.LpProblem('sample', pulp.LpMaximize)
        self.x = []
        x = []
        #
        # 反応定義
        #
        for rid in self.reactionid_array:
            lb= self.reactions[rid]["lb"]
            ub= self.reactions[rid]["ub"]
            if output == "debug":
                print(rid, lb, ub)
            self.x.append(pulp.LpVariable(rid, float(lb), float(ub), pulp.LpContinuous))
        #
        # 目的関数
        #
        coeffs = []
        rids = []
        for optstep in self.optstep:
            reactionid = self.reactions[optstep]["reactionid"]
            coef =  self.optstep[optstep]
            coeffs.append(float(coef))
            rids.append(self.x[reactionid])
            if output == "debug":
                print(coeffs, rids, reactionid)

        self.problem += pulp.lpDot(coeffs, rids)
        #self.problem += pulp.lpSum(coeffs * rids)
        #
        # 制約を追加 反応の制約
        #
        for rid in self.constraint:
            reactionid = self.reactions[rid]["reactionid"]
            value = self.constraint[rid]
            if output == "debug":
                print(rid, reactionid, value)
            self.problem +=  self.x[reactionid] == float(value)

        #
        # 制約を追加 化学量論
        #
        for metabolite in self.maboliteid_array:
            coeffs = []
            rids = []
            for rid in self.metabolites[metabolite]["reactions"]:
                reactionid = self.reactions[rid]["reactionid"]
                coeffs.append(float(self.metabolites[metabolite]["reactions"][rid]))
                rids.append(self.x[reactionid])
            if output == "debug":
                print(metabolite, coeffs, rids)
            if metabolite in self.inconstant_onlyincrease:
                self.problem += pulp.lpDot(coeffs, rids) >= 0
            elif metabolite in self.inconstant:
                #self.problem += pulp.lpDot(coeffs, rids) >= 0
                pass
            else:
                self.problem += pulp.lpDot(coeffs, rids) == 0
        #self.status = self.problem.solve(pulp.GLPK(msg=0))
        self.status = self.problem.solve()

        #
        # PuLPのモデル構築  無駄フラックス最小
        #
        self.problem2 = pulp.LpProblem('sample', pulp.LpMinimize)
        self.x2 = []
        x2 = []
        #
        # 反応定義
        #
        #for rid in self.reactionid_array:
        for i in range(len(self.reactionid_array)):
            rid = str(self.x[i])
            lb= self.reactions[rid]["lb"]
            ub= self.reactions[rid]["ub"]
            if self.x[i].value() == None:
                pass
            elif self.x[i].value() > 0:
                lb = 0.0
            elif self.x[i].value() < 0:
                ub = 0.0
            else:
                pass
            if output == "debug":
                print(rid, lb, ub)
            self.x2.append(pulp.LpVariable(rid, float(lb), float(ub), pulp.LpContinuous))
        #
        # 目的関数
        #
        coeffs = []
        rids = []
        for i in range(len(self.reactionid_array)):
            if self.x[i].value() == None:
                coef = 0.0
            elif self.x[i].value() > 0:
                coef = 1.0
            elif self.x[i].value() < 0:
                coef = -1.0
            else:
                coef = 0.0
            coeffs.append(coef)
            rids.append(self.x2[i])
            if output == "debug":
                print(self.x2[i], coeffs, rids)
        self.problem2 += pulp.lpDot(coeffs, rids)

        #for optstep in self.optstep:
        #    reactionid = self.reactions[optstep]["reactionid"]
        #    coef =  self.optstep[optstep]
        #    coeffs.append(float(coef))
        #    rids.append(self.x2[reactionid])
        #    if output == "debug":
        #        print(coeffs, rids)
        #self.problem2 += pulp.lpDot(coeffs, rids)
        #self.problem += pulp.lpSum(coeffs * rids)
        #
        # 制約を追加 反応の制約
        #
        for rid in self.constraint:
            reactionid = self.reactions[rid]["reactionid"]
            value = self.constraint[rid]
            if output == "debug":
                print(rid, reactionid, value)
            self.problem2 +=  self.x2[reactionid] == float(value)
        for i in range(len(self.reactionid_array)):
            rid = str(self.x[i])
            if rid in self.optstep:
                value = self.x[i].value()
                #print(i, rid, value)
                self.problem2 +=  self.x2[i] == float(value)

        #
        # 制約を追加 化学量論
        #
        for metabolite in self.maboliteid_array:
            coeffs = []
            rids = []
            for rid in self.metabolites[metabolite]["reactions"]:
                reactionid = self.reactions[rid]["reactionid"]
                coeffs.append(float(self.metabolites[metabolite]["reactions"][rid]))
                rids.append(self.x2[reactionid])
            if output == "debug":
                print(metabolite, coeffs, rids)
            if metabolite in self.inconstant_onlyincrease:
                self.problem2 += pulp.lpDot(coeffs, rids) >= 0
            elif metabolite in self.inconstant:
                #self.problem += pulp.lpDot(coeffs, rids) >= 0
                pass
            else:
                self.problem2 += pulp.lpDot(coeffs, rids) == 0
        #self.status2 = self.problem2.solve(pulp.GLPK(msg=0))
        self.status = self.problem2.solve()



        return(pulp.LpStatus[self.status], pulp.value(self.problem.objective))

    def show_result(self, filename = "none"):
        if filename == "none":
            for i in range(len(self.reactionid_array)):
                rid = str(self.x[i])
                value =  self.x2[i].value()
                group = self.reactions[rid]["group"]
                if value == None:
                    pass
                elif not -0.0001 < value <0.0001:
                    print(rid,"\t" ,'{:>9.4f}'.format(value),"\t", self.reactions[rid]["reaction"],"\t", str(group))
        else:
            with open(filename, mode='w') as f:
                for i in range(len(self.reactionid_array)):
                    rid = str(self.x2[i])
                    reaction = self.reactions[rid]["reaction"]
                    lb = self.reactions[rid]["lb"]
                    ub = self.reactions[rid]["ub"]
                    group = self.reactions[rid]["group"]
                    line = rid+"\t" +str(group)+"\t" +str(lb)+"\t" +str(ub)+"\t" +str(self.x2[i].value())+"\t"+"\t"+self.reactions[rid]["reaction"]+"\n"
                    f.writelines(line)

    def add_constraint(self, reaction, value = 0):
        self.constraint[reaction] = value
    def delete_constraint(self, reaction):
        del self.constraint[reaction]
    def deletiongenes(self):
        reactions = []
        for reaction in self.reactionid_array:
            if reaction in self.constraint:
                continue
            if reaction in self.optstep:
                continue
            reactions.append(reaction)
        return(reactions)
    def get_value(self, reaction):
        reactionid = self.reactions[reaction]["reactionid"]

        return(self.x2[reactionid].value())
    def single_gene_deletions(self, target):
        reactions =  self.deletiongenes()
        for reaction in  reactions:
            self.add_constraint(reaction, value = 0)
            status = self.solve()
            value = self.get_value(target)
            print(reaction,"\t" ,value,"\t" ,status[0],"\t" ,status[1])
            self.delete_constraint(reaction)
    def add_optstep(self, reaction, value = 1.0):
        self.optstep[reaction] = value
    def delete_optstep(self, reaction):
        del self.optstep[reaction]
    def clear_optstep(self):
        self.optstep = {}
    def set_boundary(self, reaction, lb, ub):
        self.reactions[reaction]["lb"] = lb
        self.reactions[reaction]["ub"] = ub
    def show_reaction(self, reaction):
        print(reaction,"\t" ,self.get_value(reaction))

    def add_inconstant_onlyincrease(self, reaction):
        if type(reaction) is not str:
            for rid in reaction:
                self.inconstant_onlyincrease[rid] = 1
        else:
            self.inconstant_onlyincrease[reaction] = 1
    def delete_inconstant_onlyincrease(self, reaction):
        if type(reaction) is not str:
            for rid in reaction:
                del self.inconstant_onlyincrease[rid]
        else:
            del self.inconstant_onlyincrease[reaction]


    def add_inconstant(self, reaction):
        if type(reaction) is not str:
            for rid in reaction:
                self.inconstant[rid] = 1
        else:
            self.inconstant[reaction] = 1
    def delete_inconstant(self, reaction):
        if type(reaction) is not str:
            for rid in reaction:
                del self.inconstant[rid]
        else:
            del self.inconstant[reaction]
    def get_metabolites(self):
        return(self.metabolites.keys())



初期値の設定

In [32]:
##################################################################################
#
#    Test code for AraGEM model
#
deffilename = "iMM904_181030aminoacid_def.txt"
modelfilename = "iMM904_181030aminoacid.txt"

model = MetabolicModel(deffilename, modelfilename)
#
#Add optimization reaction
#
#model.add_optstep("iMM904")
#
#Delete optimization reaction
#
#model.delete_optstep("iMM904")
#
#Add constraint
#
model.add_constraint("GLCt1", 10)
model.add_constraint("O2t", 0.1)
model.add_constraint("dummyprotein", 0)
#
#Delete constraint
#
#model.delete_constraint("GLCt1")#photon uptake


#
#optimize
#
status,objective = model.solve()
print("Initial condition", status, objective)
#
#
# Show result in list
#
model.show_result()
#
# Show result of one reaction
#
model.show_reaction("iMM904") # Biomass
model.show_reaction("GLYCt") # Glycerol production
model.show_reaction("ETOHt") # Ethanol production
model.show_reaction("ACt2r") #
model.show_reaction("ACtr") #
model.show_reaction("CO2t") #
#
# Save to result file
#
model.show_result("iMM904Test.txt")




Initial condition Optimal 0.21715174
iMM904 	    0.2172 	 (1.1348) 13BDglcn[c] + (0.4588) ala-L[c] + (0.046) amp[c] + (0.1607) arg-L[c] + (0.1017) asn-L[c] + (0.2975) asp-L[c] + (59.276) atp[c] + (0.000001) camp[c] + (0.000001) chitin[c] + (0.0447) cmp[c] + (0.000001) coa[c] + (0.0066) cys-L[c] + (0.0036) damp[c] + (0.0024) dcmp[c] + (0.0024) dgmp[c] + (0.0036) dtmp[c] + (0.0007) ergst[c] + (0.000001) fad[c] + (0.1054) gln-L[c] + (0.3018) glu-L[c] + (0.2904) gly[c] + (0.5185) glycogen[c] + (0.046) gmp[c] + (0.000001) gthrd[c] + (59.276) h2o[c] + (0.0663) his-L[c] + (0.1927) ile-L[c] + (0.2964) leu-L[c] + (0.2862) lys-L[c] + (0.8079) mannan[c] + (0.0507) met-L[c] + (0.000001) nad[c] + (0.000006) pa_SC[c] + (0.00006) pc_SC[c] + (0.000045) pe_SC[c] + (0.000001) pheme[m] + (0.1339) phe-L[c] + (0.1647) pro-L[c] + (0.000017) ps_SC[c] + (0.000001) thmtp[c] + (0.000053) ptd1ino_SC[c] + (0.000001) q6[m] + (0.00099) ribflv[c] + (0.1854) ser-L[c] + (0.02) so4[c] + (0.1914) thr-L[c] + (0.000066) t

ACALDtm 	   -0.0969 	 acald[m] <==> acald[c] 	 acetaldehyde mitochondrial diffusion
ACtm 	    0.1081 	 ac[c] <==> ac[m] 	 acetate transport, mitochondrial
CITtam 	   -0.1645 	 cit[c] + mal-L[m] <==> cit[m] + mal-L[c] 	 citrate transport, mitochondrial
CITtbm 	   -0.0574 	 cit[c] + pep[m] <==> cit[m] + pep[c] 	 citrate transport, mitochondrial
CO2tm 	   -0.6113 	 co2[c] <==> co2[m] 	 CO2 transport (diffusion), mitochondrial
COAtim 	    0.0029 	 coa[c] --> coa[m] 	 CoA transporter (mitochondrial), irreversible
E4Ptm 	    0.0574 	 e4p[c] <==> e4p[m] 	 D-erythrose 4-phosphate mtiochondrial transport via diffusion
ETOHtm 	   -0.0969 	 etoh[c] <==> etoh[m] 	 ethanol transport to mitochondria (diffusion)
FRDcm 	    0.0333 	 fadh2[m] + fum[c] --> fad[m] + succ[c] 	 fumarate reductase, cytosolic/mitochondrial
GLUt7m 	    0.0970 	 glu-L[c] --> glu-L[m] 	 Glutamate transport (uniporter), mitochondrial
H2Otm 	   -0.1040 	 h2o[c] <==> h2o[m] 	 H2O transport, mitochondrial
HMGCOAtm 	   -0.0029 	 hmg

動作確認①

In [33]:
print("Check aerobic anc anaerobic condition")

model.add_constraint("GLCt1", 10)
for o2i in [0.1, 1.0, 2,5,10]:
    model.add_constraint("O2t", o2i)
    status,objective = model.solve()
    print(o2i, status, objective)
    model.show_reaction("iMM904")
    model.show_reaction("GLCt1")
    model.show_reaction("O2t")
    model.show_reaction("CO2t")
    model.show_reaction("ETOHt")
    model.show_reaction("GLYCt")
    model.show_reaction("G6PDH2")
    model.show_reaction("GND")
    model.show_reaction("CSm")
    model.show_reaction("AKGDam")
    model.show_reaction("AKGDbm")
    model.show_reaction("SUCOASm")
model.add_constraint("GLCt1", 0)
model.delete_constraint("O2t")


Check aerobic anc anaerobic condition
0.1 Optimal 0.21715174
iMM904 	 0.21715174
GLCt1 	 10.0
O2t 	 0.1
CO2t 	 -17.226659
ETOHt 	 16.687826
GLYCt 	 0.51322038
G6PDH2 	 0.0
GND 	 0.00062974005
CSm 	 0.22192505
AKGDam 	 0.0
AKGDbm 	 0.0
SUCOASm 	 1.7372139e-06
1.0 Optimal 0.2642853
iMM904 	 0.2642853
GLCt1 	 10.0
O2t 	 1.0
CO2t 	 -17.626638
ETOHt 	 16.437185
GLYCt 	 0.0
G6PDH2 	 0.50780484
GND 	 0.50857126
CSm 	 0.28302392
AKGDam 	 0.0
AKGDbm 	 0.0
SUCOASm 	 2.1142824e-06
2 Optimal 0.29974049
iMM904 	 0.29974049
GLCt1 	 10.0
O2t 	 2.0
CO2t 	 -17.885468
ETOHt 	 15.6706
GLYCt 	 0.0
G6PDH2 	 0.85136682
GND 	 0.85223607
CSm 	 0.51779547
AKGDam 	 0.19680248
AKGDbm 	 0.19680248
SUCOASm 	 -0.19680008
5 Optimal 0.3929887
iMM904 	 0.3929887
GLCt1 	 10.0
O2t 	 5.0
CO2t 	 -18.81285
ETOHt 	 13.531136
GLYCt 	 0.0
G6PDH2 	 0.64066251
GND 	 0.64180218
CSm 	 1.6300026
AKGDam 	 1.2091498
AKGDbm 	 1.2091498
SUCOASm 	 -1.2091467
10 Optimal 0.54840234
iMM904 	 0.54840234
GLCt1 	 10.0
O2t 	 10.0
CO2t 	 -20.3

動作確認②

In [34]:
print("Cost normal")

model.add_constraint("GLCt1", 10)
model.add_constraint("O2t", 0.1)
status,objective = model.solve()
print(o2i, status, objective)
model.show_reaction("iMM904")
model.show_reaction("GLCt1")
model.show_reaction("O2t")
model.show_reaction("Trans1")
model.show_reaction("SO3ti")
normal = model.get_value("iMM904")
print("Cost for H2S synthesis")
model.add_constraint("GLCt1", 10)
model.add_constraint("O2t", 0.1)
model.add_constraint("Trans1", 0.2)
status,objective = model.solve()
print(o2i, status, objective)
model.show_reaction("iMM904")
model.show_reaction("GLCt1")
model.show_reaction("O2t")
model.show_reaction("Trans1")
model.show_reaction("SO3ti")
print("H2S",normal- model.get_value("iMM904"))
model.delete_constraint("Trans1")

Cost normal
10 Optimal 0.21715174
iMM904 	 0.21715174
GLCt1 	 10.0
O2t 	 0.1
Trans1 	 0.0
SO3ti 	 0.0
Cost for H2S synthesis
10 Optimal 0.21220008
iMM904 	 0.21220008
GLCt1 	 10.0
O2t 	 0.1
Trans1 	 0.2
SO3ti 	 0.0
H2S 0.004951659999999997


動作確認③

In [35]:
print("Cost for SO2 synthesis")
model.add_constraint("GLCt1", 10)
model.add_constraint("O2t", 0.1)
model.add_constraint("SO3ti", 0.2)
status,objective = model.solve()
print(o2i, status, objective)
model.show_reaction("iMM904")
model.show_reaction("GLCt1")
model.show_reaction("O2t")
model.show_reaction("Trans1")
model.show_reaction("SO3ti")
print("SO3",normal- model.get_value("iMM904"))
model.delete_constraint("SO3ti")


Cost for SO2 synthesis
10 Optimal 0.21315428
iMM904 	 0.21315428
GLCt1 	 10.0
O2t 	 0.1
Trans1 	 0.0
SO3ti 	 0.2
SO3 0.003997460000000008


動作確認④

In [44]:
print("Check protein as carbon source")

l = [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.5]
reaction_name_list = ["iMM904","GLCt1","O2t","CO2t","alcohol1",
"alcohol2","alcohol3","alcohol4","alcohol5",
"SO4ti","SO3ti","Trans1","Trans2","Trans3",
"Trans4","CYS41","CYS42","CYS31","CYS32",
"CYS43","CYS33","CYS34","ATFS2","cycteindeg2"]

for reaction_name in reaction_name_list:
    model.add_constraint(reaction_name, l[0])
    status,objective = model.solve()
    model.add_constraint(reaction_name, l[1])
    status,objective = model.solve()
    model.add_constraint(reaction_name, l[2])
    status,objective = model.solve()
    model.add_constraint(reaction_name, l[3])
    status,objective = model.solve()
    model.add_constraint(reaction_name, l[4])
    status,objective = model.solve()
    model.add_constraint(reaction_name, l[5])
    status,objective = model.solve()
    model.add_constraint(reaction_name, l[6])
    status,objective = model.solve()



Check protein as carbon source
-753515.53


In [37]:
print("Check aerobic anc anaerobic condition")

model.add_constraint("GLCt1", 0)
model.add_constraint("dummyprotein", 0)

for o2i in [0, 0.1, 1.0, 2,5,10]:
    print('■■■■■■■■■■■■■■■■')
    model.add_constraint("O2t", o2i)
    status,objective = model.solve()
    print(o2i, status, objective)
    model.show_reaction("iMM904")
    model.show_reaction("GLCt1")
    model.show_reaction("O2t")
    model.show_reaction("CO2t")
    model.show_reaction("ETOHt")
    model.show_reaction("GLYCt")
    model.show_reaction("G6PDH2")
    model.show_reaction("GND")
    model.show_reaction("CSm")
    model.show_reaction("AKGDam")
    model.show_reaction("AKGDbm")
    model.show_reaction("SUCOASm")

model.add_constraint("GLCt1", 0)
model.delete_constraint("O2t")


Check aerobic anc anaerobic condition
■■■■■■■■■■■■■■■■
0 Optimal 0.0
iMM904 	 0.0
GLCt1 	 0.0
O2t 	 0.0
CO2t 	 0.0
ETOHt 	 0.0
GLYCt 	 0.0
G6PDH2 	 0.0
GND 	 0.0
CSm 	 0.0
AKGDam 	 0.0
AKGDbm 	 0.0
SUCOASm 	 0.0
■■■■■■■■■■■■■■■■
0.1 Infeasible -0.0022996122
iMM904 	 0.0
GLCt1 	 0.0
O2t 	 0.1
CO2t 	 -28.3
ETOHt 	 0.0
GLYCt 	 0.0
G6PDH2 	 0.0
GND 	 0.0
CSm 	 0.0
AKGDam 	 24.4
AKGDbm 	 24.4
SUCOASm 	 -24.4
■■■■■■■■■■■■■■■■
1.0 Infeasible -0.022247739
iMM904 	 0.0
GLCt1 	 0.0
O2t 	 1.0
CO2t 	 -4.5800632e-13
ETOHt 	 0.0
GLYCt 	 0.0
G6PDH2 	 0.0
GND 	 0.0
CSm 	 85.666667
AKGDam 	 33.444444
AKGDbm 	 33.444444
SUCOASm 	 -33.444444
■■■■■■■■■■■■■■■■
2 Infeasible -0.044813921
iMM904 	 0.0
GLCt1 	 0.0
O2t 	 2.0
CO2t 	 -92.0
ETOHt 	 0.0
GLYCt 	 0.0
G6PDH2 	 0.0
GND 	 0.0
CSm 	 138.0
AKGDam 	 -138.0
AKGDbm 	 -138.0
SUCOASm 	 138.0
■■■■■■■■■■■■■■■■
5 Infeasible -6.9001134e-14
iMM904 	 0.0
GLCt1 	 0.0
O2t 	 5.0
CO2t 	 -193.69937
ETOHt 	 0.0
GLYCt 	 0.0
G6PDH2 	 0.0
GND 	 0.0
CSm 	 90.399707
AKGDam 	 0